<a href="https://colab.research.google.com/github/rohitha077/Music-Recommendation-and-Image-Recognition-/blob/main/rohi7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import random
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore")

def create_dummy_data(num_users=100, num_songs=200, num_ratings=1000):
    user_ids = np.random.randint(1, num_users + 1, num_ratings)
    song_ids = np.random.randint(1, num_songs + 1, num_ratings)
    ratings = np.random.randint(1, 6, num_ratings)
    data = {'user_id': user_ids, 'song_id': song_ids, 'rating': ratings}
    df = pd.DataFrame(data)

    song_meta = {'song_id': range(1, num_songs + 1),
                 'title': [f'Song_{i}' for i in range(1, num_songs + 1)],
                 'artist': [f'Artist_{i}' for i in range(1, num_songs + 1)],
                 'genre': [random.choice(['Pop', 'Rock', 'Electronic', 'Classical', 'Jazz']) for _ in range(num_songs)]}
    song_meta_df = pd.DataFrame(song_meta)
    return df, song_meta_df

df, song_meta_df = create_dummy_data()

df = pd.merge(df, song_meta_df, on='song_id', how='left')

print("Dataframe Info:")
df.info()

print("\nFirst 5 rows of the dataframe:")
print(df.head())

user_ids = df['user_id'].unique()

song_ids = df['song_id'].unique()

user_id_mapping = {user_id: index for index, user_id in enumerate(user_ids)}
song_id_mapping = {song_id: index for index, song_id in enumerate(song_ids)}

df['user_index'] = df['user_id'].map(user_id_mapping)
df['song_index'] = df['song_id'].map(song_id_mapping)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
train_df['rating'] = scaler.fit_transform(train_df[['rating']])
test_df['rating'] = scaler.transform(test_df[['rating']])

num_users = len(user_id_mapping)
num_songs = len(song_id_mapping)

def create_model(num_users, num_songs, embedding_size=50):
    user_input = layers.Input(shape=(), name='user_input')
    song_input = layers.Input(shape=(), name='song_input')

    user_embedding = layers.Embedding(num_users, embedding_size, name='user_embedding')(user_input)
    song_embedding = layers.Embedding(num_songs, embedding_size, name='song_embedding')(song_input)

    user_flatten = layers.Flatten()(user_embedding)
    song_flatten = layers.Flatten()(song_embedding)

    concatenated = layers.Concatenate()([user_flatten, song_flatten])

    dense1 = layers.Dense(128, activation='relu')(concatenated)
    dense2 = layers.Dense(64, activation='relu')(dense1)
    output = layers.Dense(1, activation='sigmoid', name='output')(dense2)

    model = keras.Model(inputs=[user_input, song_input], outputs=output)
    return model

model = create_model(num_users, num_songs)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
print("\nModel Summary:")
model.summary()

history = model.fit(
    x=[train_df['user_index'], train_df['song_index']],
    y=train_df['rating'],
    batch_size=64,
    epochs=10,
    validation_data=([test_df['user_index'], test_df['song_index']], test_df['rating']),
    verbose=1
)

def get_recommendations(user_id, model, df, song_meta_df, user_id_mapping, song_id_mapping, scaler, top_n=10):
    """
    Generates song recommendations for a given user.

    Args:
        user_id (int): The ID of the user for whom to generate recommendations.
        model: The trained recommendation model.
        df (pd.DataFrame): The original dataframe.
        song_meta_df (pd.DataFrame): DataFrame containing song metadata.
        user_id_mapping (dict): Mapping from user IDs to user indices.
        song_id_mapping (dict): Mapping from song IDs to song indices.
        scaler:  The scaler used to scale the ratings.
        top_n (int, optional): The number of top recommendations to return. Defaults to 10.

    Returns:
        pd.DataFrame: A DataFrame containing the top N recommended songs for the user,
                      with columns 'song_id', 'title', 'artist', and 'predicted_rating'.
                      Returns an empty DataFrame if the user_id is not found.
    """
    if user_id not in user_id_mapping:
        print(f"User ID {user_id} not found.")
        return pd.DataFrame()

    user_index = user_id_mapping[user_id]
    all_song_ids = song_meta_df['song_id'].unique()
    all_song_indices = [song_id_mapping[song_id] for song_id in all_song_ids if song_id in song_id_mapping]

    user_indices = np.array([user_index] * len(all_song_indices))
    song_indices = np.array(all_song_indices)

    predictions = model.predict([user_indices, song_indices]).flatten()

    predictions_original_scale = scaler.inverse_transform(predictions.reshape(-1, 1)).flatten()

    recommendation_df = pd.DataFrame({
        'song_id': [list(song_id_mapping.keys())[list(song_id_mapping.values()).index(song_index)] for song_index in all_song_indices],
        'predicted_rating': predictions_original_scale
    })

    recommendation_df = pd.merge(recommendation_df, song_meta_df, on='song_id', how='inner')

    rated_song_ids = df[df['user_id'] == user_id]['song_id'].values
    recommendation_df = recommendation_df[~recommendation_df['song_id'].isin(rated_song_ids)]

    top_recommendations = recommendation_df.sort_values(by='predicted_rating', ascending=False).head(top_n)
    return top_recommendations

user_id_to_recommend = 1
recommendations = get_recommendations(user_id_to_recommend, model, df, song_meta_df, user_id_mapping, song_id_mapping, scaler)

if not recommendations.empty:
    display(HTML(f"<h3>Top 10 Recommendations for User {user_id_to_recommend}:</h3>"))
    styled_df = recommendations.style.background_gradient(cmap='viridis').set_properties(**{'font-size': '12pt'})
    display(styled_df)
else:
    print(f"No recommendations to display for user {user_id_to_recommend}.")

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  1000 non-null   int64 
 1   song_id  1000 non-null   int64 
 2   rating   1000 non-null   int64 
 3   title    1000 non-null   object
 4   artist   1000 non-null   object
 5   genre    1000 non-null   object
dtypes: int64(3), object(3)
memory usage: 47.0+ KB

First 5 rows of the dataframe:
   user_id  song_id  rating     title      artist      genre
0        9      182       1  Song_182  Artist_182       Rock
1        8      196       3  Song_196  Artist_196       Jazz
2       60       47       2   Song_47   Artist_47  Classical
3       40      165       5  Song_165  Artist_165       Jazz
4      100      146       5  Song_146  Artist_146       Rock

Model Summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ song_input          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 50)        │      5,000 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ song_embedding      │ (None, 50)        │      9,950 │ song_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 50)        │          0 │ user_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 50)        │          0 │ song_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 100)       │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     12,928 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         65 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 36,199 (141.40 KB)

 Trainable params: 36,199 (141.40 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: 0.1302 - val_loss: 0.1271
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1168 - val_loss: 0.1275
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1167 - val_loss: 0.1292
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1079 - val_loss: 0.1379
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0844 - val_loss: 0.1611
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0771 - val_loss: 0.1764
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0748 - val_loss: 0.1813
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0664 - val_loss: 0.1847
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0629 - val_loss: 0.1843
Epoch 10/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0583 - val_loss: 0.1934
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


,song_id,predicted_rating,title,artist,genre
48,50,4.046597,Song_50,Artist_50,Pop
107,109,3.914041,Song_109,Artist_109,Jazz
52,54,3.898598,Song_54,Artist_54,Rock
121,123,3.655025,Song_123,Artist_123,Pop
148,150,3.616650,Song_150,Artist_150,Jazz
117,119,3.598804,Song_119,Artist_119,Rock
174,176,3.596195,Song_176,Artist_176,Classical
90,92,3.562187,Song_92,Artist_92,Rock
35,37,3.418462,Song_37,Artist_37,Electronic
145,147,3.391378,Song_147,Artist_147,Rock
